# Tracking Misogyny in Online Communities: A Longitudinal Analysis

## Project Overview

This notebook analyzes the correlation between misogynistic language in online communities and the rise of red-pill influencers like Andrew Tate. 

### Core Research Questions:
1. **Has misogynistic language in online communities increased over time?**
2. **Is there a measurable spike in misogynistic content correlated with the rise of Andrew Tate and similar "red-pilled" influencers?**
3. **Which types of communities (e.g., subreddit categories, YouTube channels) are most affected?**

### Data Sources:
- **Reddit**: Comments from various subreddits (r/MensRights, r/Incels, r/Feminism, r/Gaming, etc.)
- **YouTube**: Comments from Andrew Tate and related influencer videos
- **Cultural Timeline**: Key events and milestones for influencers

### Methodology:
1. Data Collection + Cleaning
2. Misogyny Detection (Hybrid: Lexicon + ML)
3. Time Series Analysis
4. Community Comparison
5. Event Correlation Analysis
6. Visualization and Reporting

---

## 1. Import Required Libraries and Define Utility Functions

First, let's import all the necessary libraries and set up our environment.

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import datetime
import time
import re
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Text processing and NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import pickle

# Statistical analysis
from scipy import stats
import statsmodels.api as sm

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Data collection APIs
import requests
import json
from bs4 import BeautifulSoup

# Add project modules to path
project_root = Path.cwd().parent
sys.path.append(str(project_root / 'src'))

# Import our custom modules
from utils.config import *
from utils.text_processing import TextProcessor, create_misogyny_lexicon
from data_collection.reddit_scraper import RedditScraper
from data_collection.youtube_scraper import YouTubeScraper
from data_collection.timeline_events import TimelineEvents, create_extended_timeline
from analysis.misogyny_detector import MisogynyDetector, create_synthetic_training_data
from analysis.time_series_analysis import TimeSeriesAnalyzer
from visualization.plotting import MisogynyVisualizer

print("✅ All libraries imported successfully!")
print(f"📁 Project root: {project_root}")
print(f"📊 Analysis will be conducted from: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Initialize key components
text_processor = TextProcessor()
timeline_events = create_extended_timeline()
misogyny_detector = MisogynyDetector()
time_series_analyzer = TimeSeriesAnalyzer()
visualizer = MisogynyVisualizer()

# Display configuration
print("🔧 Project Configuration:")
print(f"   Reddit communities: {len(REDDIT_COMMUNITIES)} categories")
print(f"   YouTube targets: {len(YOUTUBE_TARGETS)} categories")
print(f"   Timeline events: {len(timeline_events.events)} events")
print(f"   Analysis granularity: {ANALYSIS_SETTINGS['time_granularity']}")
print(f"   Normalization method: {ANALYSIS_SETTINGS['normalization_method']}")

# Show sample target communities
print("\n📋 Sample Target Communities:")
for category, communities in list(REDDIT_COMMUNITIES.items())[:3]:
    print(f"   {category}: {communities}")

## 2. Data Collection: Reddit Comments

We'll collect comments from various Reddit communities to analyze trends in misogynistic language. 

**Note**: For this demonstration, we'll use synthetic data. In a real implementation, you would:
1. Set up Reddit API credentials
2. Use the RedditScraper class to collect real data
3. Handle rate limiting and API quotas

### Target Communities:
- **Men's Rights/Red-pill**: r/MensRights, r/TheRedPill, r/MGTOW
- **Feminist**: r/Feminism, r/TwoXChromosomes
- **General**: r/AskReddit, r/politics, r/gaming

In [ ]:
# For demonstration, create synthetic Reddit data
def create_synthetic_reddit_data(n_comments=5000):
    """Create synthetic Reddit comments with realistic patterns."""
    np.random.seed(42)
    
    # Date range from 2020 to 2024
    start_date = datetime.datetime(2020, 1, 1)
    end_date = datetime.datetime(2024, 1, 1)
    
    # Generate random dates with more recent data
    date_range = (end_date - start_date).days
    random_days = np.random.exponential(scale=date_range/4, size=n_comments)
    random_days = np.clip(random_days, 0, date_range)
    dates = [end_date - datetime.timedelta(days=int(day)) for day in random_days]
    
    # Community distribution (more data from controversial communities)
    communities = ['r/MensRights', 'r/TheRedPill', 'r/Feminism', 'r/TwoXChromosomes', 
                   'r/AskReddit', 'r/politics', 'r/gaming', 'r/relationship_advice']
    community_weights = [0.2, 0.15, 0.1, 0.1, 0.2, 0.1, 0.1, 0.05]
    
    # Sample comments with varying likelihood of misogyny by community
    misogyny_rates = {
        'r/MensRights': 0.4, 'r/TheRedPill': 0.6, 'r/Feminism': 0.05, 
        'r/TwoXChromosomes': 0.02, 'r/AskReddit': 0.08, 'r/politics': 0.12,
        'r/gaming': 0.15, 'r/relationship_advice': 0.1
    }
    
    # Sample comment templates
    neutral_comments = [
        "I think this is an interesting perspective on the topic.",
        "Thanks for sharing your experience with this.",
        "This article raises some important points about society.",
        "I've been following this discussion for a while now.",
        "What are your thoughts on this development?",
        "This reminds me of a similar situation I encountered.",
        "The data seems to support this conclusion.",
        "I appreciate you taking the time to explain this.",
    ]
    
    misogynistic_comments = [
        "women are always playing the victim card these days",
        "typical female behavior, can't take responsibility",
        "she's probably just looking for attention like all of them",
        "awalt - all women are like that, hypergamous by nature",
        "women hit the wall at 30 and then wonder where all the good men went",
        "feminism has destroyed traditional family values",
        "she belongs in the kitchen, not in the workplace",
        "women can't think logically, only emotionally",
        "typical feminist propaganda trying to shame men",
        "women only want alpha chads until they need beta bux",
    ]
    
    synthetic_data = []
    
    for i in range(n_comments):
        # Select community
        community = np.random.choice(communities, p=community_weights)
        
        # Determine if comment is misogynistic based on community
        is_misogynistic = np.random.random() < misogyny_rates[community]
        
        # Select comment text
        if is_misogynistic:
            base_comment = np.random.choice(misogynistic_comments)
            # Add some variation
            variations = [" honestly", " tbh", " imo", " just saying", " facts"]
            comment = base_comment + np.random.choice(variations + [""])
        else:
            comment = np.random.choice(neutral_comments)
        
        # Add temporal trend (increasing misogyny over time, especially around events)
        date = dates[i]
        if date > datetime.datetime(2022, 6, 1):  # Around Andrew Tate's peak
            if np.random.random() < 0.3:  # 30% chance to flip to misogynistic
                if not is_misogynistic:
                    comment = np.random.choice(misogynistic_comments)
                    is_misogynistic = True
        
        synthetic_data.append({
            'comment_id': f'comment_{i}',
            'subreddit': community,
            'author': f'user_{np.random.randint(1, 1000)}',
            'body': comment,
            'score': np.random.randint(-5, 100),
            'created_utc': date,
            'category': 'mens_rights' if community in ['r/MensRights', 'r/TheRedPill'] 
                       else 'feminist' if community in ['r/Feminism', 'r/TwoXChromosomes']
                       else 'general',
            'true_misogyny': is_misogynistic  # Ground truth for validation
        })
    
    return pd.DataFrame(synthetic_data)

# Create synthetic data
print("🔄 Creating synthetic Reddit data...")
reddit_data = create_synthetic_reddit_data(5000)

print(f"✅ Created {len(reddit_data)} synthetic Reddit comments")
print(f"📅 Date range: {reddit_data['created_utc'].min()} to {reddit_data['created_utc'].max()}")
print(f"🏛️ Communities: {reddit_data['subreddit'].unique()}")
print(f"📊 True misogyny rate: {reddit_data['true_misogyny'].mean():.3f}")

# Display sample
print("\n📋 Sample Comments:")
reddit_data[['subreddit', 'body', 'true_misogyny', 'created_utc']].head()

## 3. Data Collection: YouTube Comments

Next, we'll collect comments from YouTube videos by various influencers. We'll focus on:
- **Red-pill influencers**: Andrew Tate, Fresh & Fit, Sneako
- **Feminist creators**: ContraPoints, Lindsay Ellis (for contrast)
- **Mainstream**: General content creators

**Note**: This demonstration uses synthetic data. Real implementation would use the YouTube Data API.

In [ ]:
# Create synthetic YouTube data
def create_synthetic_youtube_data(n_comments=3000):
    """Create synthetic YouTube comments."""
    np.random.seed(43)
    
    # Influencer categories with different misogyny rates
    influencers = {
        'Andrew Tate': {'category': 'red_pill', 'misogyny_rate': 0.7},
        'Fresh & Fit': {'category': 'red_pill', 'misogyny_rate': 0.6},
        'Sneako': {'category': 'red_pill', 'misogyny_rate': 0.55},
        'ContraPoints': {'category': 'feminist', 'misogyny_rate': 0.05},
        'Lindsay Ellis': {'category': 'feminist', 'misogyny_rate': 0.03},
        'PewDiePie': {'category': 'mainstream', 'misogyny_rate': 0.12},
        'MrBeast': {'category': 'mainstream', 'misogyny_rate': 0.08}
    }
    
    # Generate dates with clustering around key events
    start_date = datetime.datetime(2020, 1, 1)
    end_date = datetime.datetime(2024, 1, 1)
    
    # Key event dates (increased activity)
    event_dates = [
        datetime.datetime(2022, 6, 15),  # Tate peak virality
        datetime.datetime(2022, 8, 19),  # Tate ban
        datetime.datetime(2022, 12, 29), # Tate arrest
    ]
    
    synthetic_youtube_data = []
    
    for i in range(n_comments):
        # Select influencer
        influencer = np.random.choice(list(influencers.keys()))
        influencer_data = influencers[influencer]
        
        # Generate date with clustering around events
        if np.random.random() < 0.4:  # 40% around events
            event_date = np.random.choice(event_dates)
            days_offset = np.random.normal(0, 15)  # Within ~30 days of event
            date = event_date + datetime.timedelta(days=days_offset)
        else:
            # Random date
            random_days = np.random.randint(0, (end_date - start_date).days)
            date = start_date + datetime.timedelta(days=random_days)
        
        # Clip date to valid range
        date = max(start_date, min(end_date, date))
        
        # Determine if comment is misogynistic
        is_misogynistic = np.random.random() < influencer_data['misogyny_rate']
        
        # Generate comment text
        if is_misogynistic:
            youtube_misogynistic_comments = [
                "women are destroying western civilization",
                "tate is speaking facts about female nature",
                "these feminist creators are just mad they hit the wall",
                "women only care about money and status",
                "typical female trying to shame successful men",
                "she's just jealous of alpha males like tate",
                "women need to learn their place in society",
                "feminism is cancer, based andrew tate",
                "all women are hypergamous gold diggers",
                "this is why men are going their own way"
            ]
            comment = np.random.choice(youtube_misogynistic_comments)
        else:
            youtube_neutral_comments = [
                "interesting perspective on modern society",
                "thanks for the thoughtful analysis",
                "this video really made me think",
                "appreciate the balanced viewpoint",
                "great content as always",
                "well researched and presented",
                "this is important information to know",
                "love the production quality"
            ]
            comment = np.random.choice(youtube_neutral_comments)
        
        synthetic_youtube_data.append({
            'comment_id': f'yt_comment_{i}',
            'video_id': f'video_{influencer.replace(" ", "_").lower()}_{np.random.randint(1, 20)}',
            'channel_title': influencer,
            'author': f'yt_user_{np.random.randint(1, 500)}',
            'text': comment,
            'like_count': np.random.randint(0, 50),
            'published_at': date,
            'influencer_category': influencer_data['category'],
            'true_misogyny': is_misogynistic
        })
    
    return pd.DataFrame(synthetic_youtube_data)

# Create synthetic YouTube data
print("🔄 Creating synthetic YouTube data...")
youtube_data = create_synthetic_youtube_data(3000)

print(f"✅ Created {len(youtube_data)} synthetic YouTube comments")
print(f"📅 Date range: {youtube_data['published_at'].min()} to {youtube_data['published_at'].max()}")
print(f"📺 Channels: {youtube_data['channel_title'].unique()}")
print(f"📊 True misogyny rate: {youtube_data['true_misogyny'].mean():.3f}")

# Display sample
print("\n📋 Sample YouTube Comments:")
youtube_data[['channel_title', 'text', 'true_misogyny', 'published_at']].head()

## 4. Data Cleaning and Preprocessing

Now we'll clean and preprocess our text data to prepare it for analysis. This involves:
- Removing duplicates and invalid entries
- Filtering by text length
- Standardizing date formats
- Basic text cleaning (while preserving important terms)

In [ ]:
# Clean and preprocess the data
def clean_combined_data(reddit_df, youtube_df):
    """Clean and combine Reddit and YouTube data."""
    
    # Standardize Reddit data
    reddit_clean = reddit_df.copy()
    reddit_clean['text'] = reddit_clean['body']
    reddit_clean['date'] = pd.to_datetime(reddit_clean['created_utc'])
    reddit_clean['platform'] = 'reddit'
    reddit_clean['community'] = reddit_clean['subreddit']
    
    # Standardize YouTube data
    youtube_clean = youtube_df.copy()
    youtube_clean['date'] = pd.to_datetime(youtube_clean['published_at'])
    youtube_clean['platform'] = 'youtube'
    youtube_clean['community'] = youtube_clean['channel_title']
    youtube_clean['category'] = youtube_clean['influencer_category']
    
    # Select common columns
    common_columns = ['text', 'date', 'platform', 'community', 'category', 'true_misogyny']
    
    reddit_standardized = reddit_clean[common_columns + ['score']].copy()
    youtube_standardized = youtube_clean[common_columns + ['like_count']].copy()
    
    # Add missing columns with defaults
    reddit_standardized['like_count'] = reddit_standardized['score']
    youtube_standardized['score'] = youtube_standardized['like_count']
    
    # Combine datasets
    combined_data = pd.concat([
        reddit_standardized[common_columns + ['score']], 
        youtube_standardized[common_columns + ['score']]
    ], ignore_index=True)
    
    return combined_data

# Clean the data
print("🧹 Cleaning and combining data...")
combined_data = clean_combined_data(reddit_data, youtube_data)

# Basic cleaning using our TextProcessor
print("🔤 Processing text...")
combined_data['text_length'] = combined_data['text'].str.len()
combined_data['word_count'] = combined_data['text'].str.split().str.len()

# Filter by text length (remove very short/long texts)
min_length, max_length = 10, 500
initial_count = len(combined_data)
combined_data = combined_data[
    (combined_data['text_length'] >= min_length) & 
    (combined_data['text_length'] <= max_length)
]
filtered_count = len(combined_data)

print(f"📊 Data after cleaning:")
print(f"   Total comments: {filtered_count:,} (removed {initial_count - filtered_count:,})")
print(f"   Reddit: {len(combined_data[combined_data['platform'] == 'reddit']):,}")
print(f"   YouTube: {len(combined_data[combined_data['platform'] == 'youtube']):,}")
print(f"   Date range: {combined_data['date'].min()} to {combined_data['date'].max()}")
print(f"   Average text length: {combined_data['text_length'].mean():.1f} characters")

# Show distribution by platform and category
print("\n📈 Distribution by Platform and Category:")
platform_category_dist = combined_data.groupby(['platform', 'category']).agg({
    'text': 'count',
    'true_misogyny': 'mean'
}).round(3)
platform_category_dist.columns = ['comment_count', 'misogyny_rate']
print(platform_category_dist)

## 5. Curate and Load Misogynistic Lexicon

We'll create a comprehensive lexicon of misogynistic terms for detection. This includes:
- General derogatory terms for women
- Red-pill/manosphere specific terminology
- Incel community language
- MGTOW (Men Going Their Own Way) terms

In [ ]:
# Create comprehensive misogyny lexicon
misogyny_lexicon = create_misogyny_lexicon()

# Add additional terms specific to our research
additional_terms = {
    # Andrew Tate specific terms
    'matrix', 'escape the matrix', 'top g', 'war room',
    
    # Red-pill economics
    'smv', 'sexual market value', 'wall hitting', 'post wall',
    'branch swinging', 'monkey branching',
    
    # Additional derogatory terms
    'basic bitch', 'karen', 'simp', 'white knight',
    'virtue signaling', 'blue pill', 'red pill',
    
    # Phrases indicating misogynistic thinking
    'women logic', 'female logic', 'female privilege',
    'pussy pass', 'women and children first'
}

# Combine lexicons
extended_lexicon = misogyny_lexicon.union(additional_terms)

print(f"📖 Misogyny Lexicon Statistics:")
print(f"   Base terms: {len(misogyny_lexicon)}")
print(f"   Additional terms: {len(additional_terms)}")
print(f"   Total terms: {len(extended_lexicon)}")

# Display sample terms by category
print("\n🔍 Sample Terms by Category:")

categories = {
    'Derogatory terms': ['bitch', 'slut', 'whore', 'thot', 'skank'],
    'Red-pill terms': ['hypergamy', 'awalt', 'chad', 'beta', 'alpha'],
    'Incel terms': ['femoid', 'foid', 'roastie', 'becky', 'stacy'],
    'MGTOW terms': ['gynocentrism', 'simp', 'white knight', 'pussy pass'],
    'Phrases': ['women logic', 'female privilege', 'escape the matrix']
}

for category, terms in categories.items():
    available_terms = [term for term in terms if term in extended_lexicon]
    print(f"   {category}: {available_terms[:5]}")

# Test lexicon on sample texts
test_texts = [
    "women are hypergamous by nature",
    "typical female behavior right there",
    "she's just another basic bitch seeking attention",
    "this is a normal discussion about gender",
    "Andrew Tate speaks the truth about the matrix"
]

print("\n🧪 Lexicon Testing:")
for text in test_texts:
    matching_terms = [term for term in extended_lexicon if term.lower() in text.lower()]
    print(f"   '{text[:40]}...' → {len(matching_terms)} terms: {matching_terms[:3]}")

## 6. Misogyny Detection: Hybrid Approach

We'll use our hybrid misogyny detection system that combines:
1. **Lexicon-based detection**: Pattern matching with our curated terms
2. **Machine learning classification**: Trained classifier for context understanding

This approach provides both precision (lexicon) and recall (ML model).

In [ ]:
# Train the misogyny detection model
print("🤖 Training Misogyny Detection Model...")

# Create training data (combining synthetic labeled data with our ground truth)
training_data = create_synthetic_training_data()

# Add some examples from our actual data for more realistic training
sample_data = combined_data.sample(200, random_state=42)
additional_training = pd.DataFrame({
    'text': sample_data['text'],
    'is_misogynistic': sample_data['true_misogyny'].astype(int)
})

# Combine training datasets
full_training_data = pd.concat([training_data, additional_training], ignore_index=True)
print(f"📚 Training data: {len(full_training_data)} examples")
print(f"   Positive examples: {full_training_data['is_misogynistic'].sum()}")
print(f"   Negative examples: {(full_training_data['is_misogynistic'] == 0).sum()}")

# Train the model
training_results = misogyny_detector.train_classifier(full_training_data)

print("✅ Model Training Complete!")
print(f"   Training Accuracy: {training_results['train_accuracy']:.3f}")
print(f"   Test Accuracy: {training_results['test_accuracy']:.3f}")
print(f"   Cross-validation: {training_results['cv_mean']:.3f} ± {training_results['cv_std']:.3f}")

# Apply detection to our full dataset
print("\n🔍 Applying Misogyny Detection to Full Dataset...")
detection_results = misogyny_detector.analyze_dataset(combined_data, 'text')

print(f"📊 Detection Results:")
print(f"   Detected misogynistic comments: {detection_results['is_misogynistic'].sum():,}")
print(f"   Detection rate: {detection_results['is_misogynistic'].mean():.3f}")
print(f"   Average confidence: {detection_results['confidence'].mean():.3f}")

# Compare with ground truth for validation
if 'true_misogyny' in detection_results.columns:
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    y_true = detection_results['true_misogyny']
    y_pred = detection_results['is_misogynistic']
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"\n✅ Validation Against Ground Truth:")
    print(f"   Accuracy: {accuracy:.3f}")
    print(f"   Precision: {precision:.3f}")
    print(f"   Recall: {recall:.3f}")
    print(f"   F1-Score: {f1:.3f}")

# Show examples of detected misogyny
print("\n📋 Sample Detected Misogynistic Comments:")
misogynistic_samples = detection_results[detection_results['is_misogynistic']].nlargest(5, 'confidence')
for idx, row in misogynistic_samples.iterrows():
    print(f"   Platform: {row['platform']}, Confidence: {row['confidence']:.3f}")
    print(f"   Text: {row['text'][:100]}...")
    print()

## 7. Time Series Analysis: Tracking Trends Over Time

Now we'll analyze how misogynistic content has changed over time and correlate it with key events in the red-pill influencer timeline.

In [ ]:
# Prepare time series data
print("📈 Preparing Time Series Analysis...")

# Create time series aggregation
time_series_data = time_series_analyzer.prepare_time_series(
    detection_results, 
    date_column='date',
    misogyny_column='is_misogynistic',
    community_column='platform'
)

print(f"📊 Time Series Data:")
print(f"   Data points: {len(time_series_data)}")
print(f"   Date range: {time_series_data['date'].min()} to {time_series_data['date'].max()}")
print(f"   Platforms: {time_series_data['platform'].unique() if 'platform' in time_series_data.columns else 'Combined'}")

# Calculate overall trend
overall_trend = time_series_analyzer.calculate_trend(time_series_data)

print(f"\n📈 Overall Trend Analysis:")
print(f"   Direction: {overall_trend['trend_direction']}")
print(f"   Slope: {overall_trend['slope']:.6f}")
print(f"   R-squared: {overall_trend['r_squared']:.3f}")
print(f"   Statistical significance: {'Yes' if overall_trend['is_significant'] else 'No'} (p={overall_trend['p_value']:.3f})")
print(f"   Percentage change: {overall_trend['percentage_change']:.1f}%")

# Analyze event correlations
print(f"\n🎯 Analyzing Event Correlations...")
event_correlation = time_series_analyzer.analyze_event_correlation(time_series_data)

print(f"📅 Event Correlation Results:")
print(f"   Total events analyzed: {len(event_correlation)}")

# Show significant events
significant_events = event_correlation[event_correlation['is_significant']]
if len(significant_events) > 0:
    print(f"   Significant correlations: {len(significant_events)}")
    print(f"\n🔍 Top Significant Event Impacts:")
    top_events = significant_events.nlargest(3, 'effect_size')
    for _, event in top_events.iterrows():
        print(f"   • {event['event'][:60]}...")
        print(f"     Date: {event['event_date'].strftime('%Y-%m-%d')}")
        print(f"     Change: {event['percent_change']:+.1f}%")
        print(f"     P-value: {event['p_value']:.3f}")
        print()
else:
    print("   No statistically significant event correlations found")

# Compare platforms
if 'platform' in detection_results.columns:
    print(f"\n🏛️ Platform Comparison Analysis...")
    platform_comparison = time_series_analyzer.compare_communities(
        detection_results,
        community_column='platform',
        date_column='date',
        misogyny_column='is_misogynistic'
    )
    
    print("📊 Platform Comparison Results:")
    for _, platform in platform_comparison.iterrows():
        print(f"   {platform['community']}:")
        print(f"     Misogyny rate: {platform['misogyny_rate']:.3f}")
        print(f"     Trend: {platform['trend_direction']}")
        print(f"     Comments: {platform['total_comments']:,}")

# Display time series data sample
print(f"\n📋 Sample Time Series Data:")
print(time_series_data.head())

## 8. Visualization: Creating Interactive Charts and Dashboards

Let's create comprehensive visualizations to illustrate our findings.

In [ ]:
# Create comprehensive visualizations
print("📊 Creating Visualizations...")

# 1. Time Series Plot with Events
print("   Creating time series plot...")
ts_fig = visualizer.plot_time_series(
    time_series_data,
    value_column='normalized_misogyny',
    title='Misogyny Trends Over Time with Key Events',
    include_events=True
)
ts_fig.show()

# 2. Platform/Community Comparison
if len(platform_comparison) > 0:
    print("   Creating platform comparison plot...")
    platform_fig = visualizer.plot_community_comparison(
        platform_comparison,
        title='Misogyny Rates by Platform'
    )
    platform_fig.show()

# 3. Event Correlation Analysis
if len(event_correlation) > 0:
    print("   Creating event correlation plot...")
    event_fig = visualizer.plot_event_correlation(
        event_correlation,
        title='Impact of Key Events on Misogynistic Content'
    )
    event_fig.show()

# 4. Word Frequency Analysis
print("   Analyzing most common misogynistic terms...")
misogyny_patterns = misogyny_detector.get_misogynistic_patterns(
    detection_results, 
    text_column='text'
)

if misogyny_patterns['most_common_words']:
    word_freq_fig = visualizer.plot_word_frequency(
        misogyny_patterns['most_common_words'],
        title='Most Common Words in Misogynistic Comments'
    )
    word_freq_fig.show()

# 5. Comprehensive Dashboard
print("   Creating comprehensive dashboard...")
dashboard_fig = visualizer.create_dashboard(
    time_series_data,
    platform_comparison if len(platform_comparison) > 0 else None,
    event_correlation if len(event_correlation) > 0 else None,
    misogyny_patterns['most_common_words']
)
dashboard_fig.show()

# Display key statistics
print(f"\n📈 Key Visualization Insights:")
print(f"   📊 Total data points visualized: {len(time_series_data):,}")
print(f"   📅 Analysis period: {time_series_data['date'].min().strftime('%Y-%m-%d')} to {time_series_data['date'].max().strftime('%Y-%m-%d')}")
print(f"   🎯 Events with significant impact: {len(significant_events) if len(significant_events) > 0 else 0}")
print(f"   🏛️ Platforms analyzed: {len(platform_comparison) if len(platform_comparison) > 0 else 1}")
print(f"   🔤 Most common misogynistic terms: {len(misogyny_patterns['most_common_words'])}")

## 9. Final Analysis and Conclusions

Let's summarize our findings and generate a comprehensive report.

In [ ]:
# Generate comprehensive final report
print("📋 Generating Final Analysis Report...")

# Generate summary report
summary_report = time_series_analyzer.generate_summary_report(
    time_series_data,
    platform_comparison if len(platform_comparison) > 0 else None,
    event_correlation if len(event_correlation) > 0 else None
)

print(summary_report)

# Answer our core research questions
print("\n" + "="*60)
print("🎯 ANSWERS TO CORE RESEARCH QUESTIONS")
print("="*60)

print("\n1️⃣ HAS MISOGYNISTIC LANGUAGE INCREASED OVER TIME?")
if overall_trend['trend_direction'] == 'increasing':
    print(f"   ✅ YES - There is an {overall_trend['trend_direction']} trend")
    print(f"   📈 Slope: {overall_trend['slope']:.6f}")
    print(f"   📊 Percentage change: {overall_trend['percentage_change']:+.1f}%")
    print(f"   🎯 Statistical significance: {'Strong' if overall_trend['is_significant'] else 'Weak'}")
else:
    print(f"   ❌ NO - The trend is {overall_trend['trend_direction']}")
    print(f"   📈 Percentage change: {overall_trend['percentage_change']:+.1f}%")

print("\n2️⃣ IS THERE CORRELATION WITH RED-PILL INFLUENCER EVENTS?")
if len(significant_events) > 0:
    print(f"   ✅ YES - Found {len(significant_events)} significant correlations")
    print("   🔍 Key impactful events:")
    for _, event in significant_events.head(3).iterrows():
        print(f"      • {event['event'][:50]}... ({event['percent_change']:+.1f}% change)")
else:
    print("   ❌ NO - No statistically significant correlations found")
    print("   📝 This could indicate either:")
    print("      • Events don't significantly impact misogyny levels")
    print("      • Effects are delayed or indirect")
    print("      • Sample size or time window limitations")

print("\n3️⃣ WHICH COMMUNITIES ARE MOST AFFECTED?")
if len(platform_comparison) > 0:
    highest_platform = platform_comparison.iloc[0]
    print(f"   🏆 Highest misogyny rate: {highest_platform['community']}")
    print(f"   📊 Rate: {highest_platform['misogyny_rate']:.3f}")
    print(f"   📈 Trend: {highest_platform['trend_direction']}")
    
    print("\n   📋 Full platform ranking:")
    for i, (_, platform) in enumerate(platform_comparison.iterrows(), 1):
        print(f"      {i}. {platform['community']}: {platform['misogyny_rate']:.3f} rate "
              f"({platform['trend_direction']} trend)")

# Additional insights
print("\n" + "="*60)
print("💡 ADDITIONAL INSIGHTS")
print("="*60)

print(f"\n📊 DETECTION MODEL PERFORMANCE:")
print(f"   🎯 Accuracy: {accuracy:.3f}")
print(f"   🎯 Precision: {precision:.3f}")
print(f"   🎯 Recall: {recall:.3f}")
print(f"   🎯 F1-Score: {f1:.3f}")

print(f"\n🔤 LEXICON ANALYSIS:")
print(f"   📖 Terms in lexicon: {len(extended_lexicon)}")
print(f"   🎯 Most common misogynistic terms found:")
for term, count in list(misogyny_patterns['most_common_lexicon_terms'].items())[:5]:
    print(f"      • '{term}': {count} occurrences")

print(f"\n📈 DATA VOLUME:")
print(f"   💬 Total comments analyzed: {len(detection_results):,}")
print(f"   🚨 Misogynistic comments detected: {detection_results['is_misogynistic'].sum():,}")
print(f"   📊 Overall misogyny rate: {detection_results['is_misogynistic'].mean():.3f}")

# Recommendations for stakeholders
print("\n" + "="*60)
print("📝 RECOMMENDATIONS FOR STAKEHOLDERS")
print("="*60)

recommendations = [
    "🛡️ PLATFORM MODERATION: Focus resources on communities with highest misogyny rates",
    "📊 CONTINUOUS MONITORING: Implement real-time tracking of misogynistic language trends",
    "🎯 EVENT-BASED INTERVENTIONS: Prepare response strategies around influencer milestones",
    "🤝 COMMUNITY ENGAGEMENT: Develop counter-messaging campaigns in affected communities",
    "📚 RESEARCH EXPANSION: Extend analysis to more platforms and longer time periods",
    "🔍 GRANULAR ANALYSIS: Investigate specific types of misogynistic language for targeted interventions"
]

for recommendation in recommendations:
    print(f"   {recommendation}")

print(f"\n" + "="*60)
print("✅ ANALYSIS COMPLETE")
print("="*60)
print(f"📁 All results and visualizations have been saved to the project directory")
print(f"📊 Dashboard and plots are ready for presentation")
print(f"📋 This analysis provides evidence-based insights for policy and intervention decisions")